In [ ]:
from ratdata import data_manager as dm, process, ingest, plot as rdplot
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
beta_low = 11
beta_high = 30

In [ ]:
for rat in dm.Rat.select():
    print(f"{rat.full_label}")
    spectra = []
    ff = None
    for r in dm.RecordingFile.select().where(
        (dm.RecordingFile.condition == 'baseline') & (dm.RecordingFile.rat == rat)):
            filename = Path(r.dirname) / r.filename
            print(f"  {r.filename}")
            rec = ingest.read_mce_matlab_file(filename)
            if dm.is_recording_rejected(filename.name):
                print("REJECTED")
                continue
            fs = int(1/rec.dt)
            x = np.mean(rec.electrode_data, 0)
            if dm.is_recording_sliced(r.filename):
                slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
                start = int(slice.start * fs)
                end = int((slice.start + slice.length) * fs)
                x = x[start:end]
            f, psd = signal.welch(x, fs, nperseg=int(2*fs))
            m, b = process.fit_oof(f, psd, 2, 100)
            f[0] = 1e-20
            oof = f ** m * (np.e ** b)
            detrended = psd - oof
            idx = np.logical_and(f >= 10, f <= 35)
            if ff is None:
                ff = f[idx]
            spectra.append(detrended[idx])
    fig = plt.figure(figsize=(12, 6))
    avg = np.mean(np.array(spectra), 0)
    std = np.std(np.array(spectra), 0)
    plt.plot(ff, np.mean(np.array(spectra), 0), linewidth=5, color=rdplot.sham_ohda_palette[rat.full_label])
    plt.fill_between(ff, avg-std, avg+std, facecolor=rdplot.sham_ohda_palette[rat.full_label], alpha=0.4)
    plt.axhline(0, color='red')
    for s in spectra:
        plt.plot(ff, s, color=(0.5, 0.5, 0.5), linewidth=0.3)
    plt.ylim([-1e-4, 1e-4])
    plt.title(f"{rat.full_label}")
    plt.savefig(f"plots/peaks/averaged_{rat.full_label}.svg", bbox_inches="tight")
    plt.savefig(f"plots/peaks/averaged_vec_{rat.full_label}.png", bbox_inches="tight")
    